In [51]:
import pandas as pd
import folium

import seaborn as sns
import matplotlib.pyplot as plt

In [52]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
school = pd.read_csv('../data/schoolinfo.csv')
subway = pd.read_csv('../data/subwayInfo.csv')
park = pd.read_csv('../data/parkInfo.csv')
interest = pd.read_csv('../data/interestRate.csv')

In [53]:
train['contract_date'] = pd.to_datetime(train['contract_year_month'], format='%Y%m')

# 연도와 월 변수 생성
train['contract_year'] = train['contract_date'].dt.year
train['contract_month'] = train['contract_date'].dt.month

# 결과 확인
print(train[['contract_year_month', 'contract_date', 'contract_year', 'contract_month']].head())

   contract_year_month contract_date  contract_year  contract_month
0               201906    2019-06-01           2019               6
1               202003    2020-03-01           2020               3
2               202003    2020-03-01           2020               3
3               201907    2019-07-01           2019               7
4               201904    2019-04-01           2019               4


In [54]:
train2 = train.merge(interest[['year_month', 'interest_rate']], 
                    left_on='contract_year_month', 
                    right_on='year_month', 
                    how='left')

train2.drop(columns=['year_month'], inplace=True)


print(train2.head())

   index  area_m2  contract_year_month  contract_day  contract_type  floor  \
0      0  84.9981               201906            25              2      9   
1      1  84.9981               202003            26              2     20   
2      2  84.9981               202003            28              2      8   
3      3  59.3400               201907            15              2      1   
4      4  59.8100               201904            12              2      6   

   built_year   latitude   longitude  age  deposit contract_date  \
0        2019  37.054314  127.045216    0  17000.0    2019-06-01   
1        2019  37.054314  127.045216    1  23000.0    2020-03-01   
2        2019  37.054314  127.045216    1  23000.0    2020-03-01   
3        1986  36.964647  127.055847   33   5000.0    2019-07-01   
4        1995  36.972390  127.084514   24   1800.0    2019-04-01   

   contract_year  contract_month  interest_rate  
0           2019               6           1.78  
1           2020      

In [55]:
train2[['latitude','longitude']].describe()

,latitude,longitude
count,1.801228e+06,1.801228e+06
mean,3.747721e+01,1.269662e+02
std,1.625113e-01,1.701221e-01
min,3.691791e+01,1.264787e+02
25%,3.737976e+01,1.268422e+02
50%,3.750227e+01,1.269976e+02
75%,3.758205e+01,1.270888e+02
max,3.818194e+01,1.276609e+02


In [56]:
subway.describe()

,latitude,longitude
count,700.000000,700.000000
mean,37.511467,126.979502
std,0.151368,0.181557
min,36.769502,126.476241
25%,37.474883,126.880270
50%,37.527585,127.007471
75%,37.579632,127.070794
max,37.948100,127.723792


In [57]:
greater_Seoul_school = school[
    (school['latitude'] >= 36.76950200) & (school['longitude'] <= 127.72379200)
].reset_index(drop=True)
greater_Seoul_school

,schoolLevel,latitude,longitude
0,elementary,37.703889,127.540156
1,elementary,37.676874,127.600664
2,middle,37.486153,127.056121
3,high,37.395716,126.909033
4,high,37.559989,126.853133
...,...,...,...
4780,elementary,37.477130,127.149414
4781,high,37.847992,126.881304
4782,high,37.534932,127.227443
4783,elementary,37.309024,127.089889


In [58]:
greater_Seoul_park = park[
    (park['latitude'] >= 36.76950200) & (park['longitude'] <= 127.72379200)
].reset_index(drop=True)
greater_Seoul_park

,latitude,longitude,area
0,37.509628,127.628406,856.0
1,37.508443,127.627414,847.0
2,37.493844,127.509326,1276.0
3,37.496021,127.408216,3300.0
4,37.496164,127.412326,394.0
...,...,...,...
8290,37.510749,126.809895,3660.9
8291,37.507671,126.812420,1531.3
8292,37.507496,126.811025,2455.7
8293,37.511053,126.799506,4168.1


In [59]:
# ['latitude', 'longitude']별로 그룹화
df = train2.groupby(['latitude', 'longitude']).agg(
    count=('latitude', 'size'),  # 거래 횟수
    avg_deposit=('deposit', 'mean')  # 평균 deposit 계산
).reset_index()

df_sorted = df.sort_values(by='avg_deposit', ascending=False)

pd.set_option('display.max_rows', None)

print(df_sorted)

pd.reset_option('display.max_rows')


        latitude   longitude  count    avg_deposit
11319  37.531393  127.011496      1  950000.000000
10676  37.525956  127.054237      4  900000.000000
10861  37.527616  127.047125      1  700000.000000
10817  37.527271  127.047778      4  630000.000000
11715  37.536441  127.003471     23  603021.739130
10621  37.525342  127.053412      1  580000.000000
10733  37.526497  127.054327      1  450000.000000
10539  37.524544  127.055193      5  440000.000000
10933  37.528263  127.046282      4  432500.000000
8861   37.501576  127.068106      1  430000.000000
7317   37.487864  127.043825      2  425000.000000
10245  37.520939  127.050459      1  400000.000000
12345  37.544155  127.043358     43  393872.093023
9909   37.516812  127.051689      4  384375.000000
10772  37.526829  127.054212      1  380000.000000
11333  37.531551  127.009903      3  370000.000000
13419  37.560006  127.007474      4  364750.000000
12464  37.545726  127.042333     20  359475.000000
10724  37.526417  127.048116   

In [60]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree

# Haversine 공식 함수 정의 (두 지점 간의 거리 계산)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반경 (단위: km)
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # 결과를 km 단위로 반환

# KD-Tree를 사용해 가장 가까운 지하철을 찾는 함수
def find_nearest_subway(lat, lon, subway_tree, subway_coordinates):
    # 주어진 좌표에 대해 가장 가까운 지하철 역 인덱스 찾기
    distance, index = subway_tree.query([lat, lon], k=1)
    
    # 가장 가까운 지하철 역의 좌표
    nearest_subway = subway_coordinates[index]
    
    # Haversine 공식을 사용하여 거리 계산
    dist_km = haversine(lat, lon, nearest_subway[0], nearest_subway[1])
    
    return dist_km

# subway 데이터의 좌표를 KD-Tree로 변환
subway_coordinates = subway[['latitude', 'longitude']].values
subway_tree = cKDTree(subway_coordinates)

# df_sorted의 각 latitude, longitude에 대해 가장 가까운 지하철과의 거리를 계산하여 새로운 열 추가
df_sorted['nearest_subway_distance_km'] = df_sorted.apply(
    lambda row: find_nearest_subway(row['latitude'], row['longitude'], subway_tree, subway_coordinates),
    axis=1
)

pd.set_option('display.max_rows', None)
# 결과 확인
print(df_sorted[['latitude', 'longitude', 'nearest_subway_distance_km']])
pd.reset_option('display.max_rows')

        latitude   longitude  nearest_subway_distance_km
11319  37.531393  127.011496                    0.299607
10676  37.525956  127.054237                    0.737033
10861  37.527616  127.047125                    0.581834
10817  37.527271  127.047778                    0.638890
11715  37.536441  127.003471                    0.386709
10621  37.525342  127.053412                    0.664668
10733  37.526497  127.054327                    0.797749
10539  37.524544  127.055193                    0.598377
10933  37.528263  127.046282                    0.516265
8861   37.501576  127.068106                    0.588760
7317   37.487864  127.043825                    0.279059
10245  37.520939  127.050459                    0.309249
12345  37.544155  127.043358                    0.133141
9909   37.516812  127.051689                    0.319479
10772  37.526829  127.054212                    0.833375
11333  37.531551  127.009903                    0.244604
13419  37.560006  127.007474   

In [61]:
# 보행 속도: 1km 당 12분
walking_speed_per_km = 12

# 걸리는 시간 계산 (거리 * 12)
df_sorted['walking_time_min'] = df_sorted['nearest_subway_distance_km'] * walking_speed_per_km


In [62]:
df_sorted

,latitude,longitude,count,avg_deposit,nearest_subway_distance_km,walking_time_min
11319,37.531393,127.011496,1,950000.00000,0.299607,3.595290
10676,37.525956,127.054237,4,900000.00000,0.737033,8.844394
10861,37.527616,127.047125,1,700000.00000,0.581834,6.982011
10817,37.527271,127.047778,4,630000.00000,0.638890,7.666678
11715,37.536441,127.003471,23,603021.73913,0.386709,4.640509
...,...,...,...,...,...,...
17427,37.683331,126.604119,1,1035.00000,10.042994,120.515931
15996,37.618899,127.077603,1,1000.00000,0.241247,2.894963
4266,37.419602,126.907851,1,1000.00000,0.086014,1.032170
4210,37.415485,126.915953,1,1000.00000,0.763623,9.163476
